In [ ]:
import requests

url = 'https://api.notion.com/v1/pages'
headers = {
    'Authorization': f'Bearer {NOTION_API_KEY}',
    'Notion-Version': '2022-06-28',
    'Content-Type': 'application/json',
}

json_data = {
    "parent": { "database_id": NOTION_DB_ID },
    "properties": {
        # データベースのタイトルプロパティ名で指定
        "名前": {
            "title": [
                { "text": { "content": "APIテスト" } }
            ]
        },
        # 例：セレクトプロパティ
        "タグ": {
            "multi_select": [
                { "name": "notion_api" }
            ]
        },
        "Progress": {
            "status": {
                "name": "未読"
            }
        },
        "追加日": {
            "date": {
                "start": "2025-09-20",
                "end": None,
                "time_zone": "Asia/Tokyo"
            }
        }
    },
} 

response = requests.post(url, headers=headers, json=json_data)
print(response.json())
